In [9]:
#use xarray package to create new netcdfs for pattern correlation
#compare Siberia 2020 Heatwave to model data to find pattern corr. Lowered resolution of ERA5 to allow comparison to model.

#generate excel sheet of pattern correlations for each EC earth model day against Siberia 2020 heatwave

scenario = 'ssp126'

##IMPORT PACKAGES
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp
import cartopy.feature as cfeature ## for lakes/borders etc
countries_50m = cfeature.NaturalEarthFeature('cultural','admin_0_countries','50m',edgecolor='k',facecolor='none')
import cartopy.crs as ccrs
import numpy as np
from netCDF4 import Dataset  
import xarray as xr
import datetime as dt
import pandas as pd

import matplotlib.pyplot as plt
import math
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import os
import iris
from iris.experimental.equalise_cubes import equalise_attributes
from iris.util import unify_time_units
import iris.coord_categorisation
import iris.plot as iplt
import iris.quickplot as qplt
import iris.analysis.stats
import pandas as pd
import seaborn as sns
import glob
import datetime
import cf_units as unit

crs = ccrs.PlateCarree()

import warnings
warnings.filterwarnings("ignore")

In [10]:
date_string_list = ['2015-06-29','2032-06-28','2045-06-24','2063-06-28','2079-06-27','2082-06-30','2090-06-26','2091-06-26','2092-06-30','2094-06-30','2095-06-26','2096-06-27','2098-06-26','2098-06-26','2099-06-24','2099-06-29','2099-06-26','2015-07-27','2016-07-24','2029-07-21','2035-07-22','2036-07-23','2038-07-24','2040-07-16','2048-07-13','2049-07-03','2050-07-09','2051-07-13','2052-07-22','2054-07-25','2058-07-04','2059-07-14','2060-07-10','2061-07-30','2061-07-04','2062-07-19','2062-07-07','2063-07-10','2063-07-14','2066-07-14','2067-07-16','2068-07-13','2071-07-19','2071-07-27','2071-07-22','2072-07-30','2073-07-07','2074-07-17','2075-07-20','2076-07-25','2076-07-12','2076-07-11','2076-07-07','2077-07-26','2077-07-23','2078-07-21','2078-07-15','2078-07-17','2079-07-01','2079-07-07','2081-07-09','2081-07-01','2082-07-09','2082-07-19','2082-07-07','2082-07-30','2086-07-08','2086-07-07','2086-07-18','2086-07-02','2086-07-09','2086-07-05','2087-07-01','2087-07-24','2087-07-08','2089-07-10','2090-07-15','2090-07-10','2091-07-13','2091-07-24','2092-07-07','2093-07-13','2093-07-23','2094-07-06','2094-07-29','2094-07-30','2094-07-22','2094-07-19','2095-07-31','2095-07-17','2095-07-17','2096-07-01','2096-07-14','2096-07-23','2096-07-31','2096-07-11','2096-07-10','2097-07-13','2098-07-28','2098-07-17','2098-07-15','2099-07-15','2099-07-14','2099-07-08','2099-07-20','2099-07-11','2099-07-27','2100-07-02','2100-07-08','2100-07-31','2100-07-21']
member_list = ['r112i1p1f1','r111i1p1f1','r105i1p1f1','r101i1p1f1','r103i1p1f1','r108i1p1f1','r108i1p1f1','r101i1p1f1','r107i1p1f1','r103i1p1f1','r112i1p1f1','r105i1p1f1','r105i1p1f1','r108i1p1f1','r102i1p1f1','r103i1p1f1','r110i1p1f1','r108i1p1f1','r107i1p1f1','r111i1p1f1','r109i1p1f1','r112i1p1f1','r111i1p1f1','r102i1p1f1','r106i1p1f1','r108i1p1f1','r109i1p1f1','r111i1p1f1','r110i1p1f1','r105i1p1f1','r105i1p1f1','r110i1p1f1','r101i1p1f1','r107i1p1f1','r111i1p1f1','r105i1p1f1','r106i1p1f1','r101i1p1f1','r111i1p1f1','r109i1p1f1','r107i1p1f1','r109i1p1f1','r107i1p1f1','r109i1p1f1','r112i1p1f1','r105i1p1f1','r105i1p1f1','r109i1p1f1','r105i1p1f1','r101i1p1f1','r102i1p1f1','r108i1p1f1','r109i1p1f1','r106i1p1f1','r110i1p1f1','r109i1p1f1','r110i1p1f1','r111i1p1f1','r103i1p1f1','r104i1p1f1','r101i1p1f1','r108i1p1f1','r107i1p1f1','r108i1p1f1','r109i1p1f1','r110i1p1f1','r101i1p1f1','r102i1p1f1','r103i1p1f1','r105i1p1f1','r111i1p1f1','r112i1p1f1','r109i1p1f1','r110i1p1f1','r112i1p1f1','r106i1p1f1','r101i1p1f1','r105i1p1f1','r102i1p1f1','r108i1p1f1','r107i1p1f1','r102i1p1f1','r104i1p1f1','r103i1p1f1','r105i1p1f1','r107i1p1f1','r109i1p1f1','r111i1p1f1','r104i1p1f1','r109i1p1f1','r111i1p1f1','r102i1p1f1','r103i1p1f1','r104i1p1f1','r105i1p1f1','r108i1p1f1','r112i1p1f1','r101i1p1f1','r104i1p1f1','r106i1p1f1','r108i1p1f1','r103i1p1f1','r106i1p1f1','r107i1p1f1','r108i1p1f1','r110i1p1f1','r112i1p1f1','r101i1p1f1','r106i1p1f1','r107i1p1f1','r108i1p1f1']

In [11]:
def sorted(cube, coord):
    coord_to_sort = cube.coord(coord)
    assert coord_to_sort.ndim == 1, 'One dim coords only please.'
    dim, = cube.coord_dims(coord_to_sort)
    index = [slice(None)] * cube.ndim
    index[dim] = np.argsort(coord_to_sort.points)
    return cube[tuple(index)]

In [12]:
#inputs
#membername = 'r109i1p1f1'

#Lat long boundaries
#Specify min and max values of latitude and longitude 
lat_min = 50
lat_max =90
lon_min = 60
lon_max = 180 

path = '/nfs/annie/earamay/Prash/'+scenario+'/'

fn2 = '/nfs/annie/ee21pnm/1988_pcorr_small.nc'
set_date = dt.date(1988,7,25)


output_name = "/nfs/see-fs-01_teaching/ee21pnm/Spreadsheets/Siberia_Corr_"+scenario+"_1988_07_25_v2.xlsx"

#save histograms
pathout ="/nfs/see-fs-01_teaching/ee21pnm/Project/output/Pattern_Corr_Histograms/Siberia"

In [13]:
#load control day cube (eg 28 June 2021)
cube1 = iris.load_cube(fn2)

longitude = cube1.coord('longitude')
latitude = cube1.coord('latitude')

# Functions to identify latitudes and longitudes we want to subset to
#def region_lat(input):
#    return lat_min  <= input <= lat_max 

#def region_long(input):
#    return lon_min  <= input <= lon_max 

#region_con = iris.Constraint(latitude = region_lat, longitude = region_long)

# Subset the cube to the location of interest
#region_subset = cube1.extract(region_con)

#find spatial average for each day
coords = ('longitude', 'latitude')

#find cosine latitude weighting for varying area of earth grid
cosinelat_weights=iris.analysis.cartography.cosine_latitude_weights(cube1)

In [14]:
#START PATTERN CORR

p_corr_list = []

count = 0
for date_string in date_string_list:
    #date_text = custom_strftime('{S} %B %Y', dt.datetime.strptime(date_string, '%Y-%m-%d'))
    year = int(date_string[0:4])
    model_month = int(date_string[5:7])
    model_day = int(date_string[8:10])

    #load model day

    #date
    model_day2 = datetime.datetime(year, model_month, model_day,12)
    
    #specify member
    membername = member_list[count]

    ##Read in geopotential data
    zfile = glob.glob(path + "Z/zg_day_EC-Earth3_"+scenario+"_" + membername + "_gr_" +str(year)+ "0101-" +str(year)+ "1231.nc")
    fn = zfile[0]
    cube2 = iris.load_cube(fn)

    longitude2 = cube2.coord('longitude')
    latitude2 = cube2.coord('latitude')
    pressure2 = cube2.coord('air_pressure')


    # Functions to identify latitudes and longitudes we want to subset to
    def region_lat(input):
        return lat_min  <= input <= lat_max 

    def region_long(input):
        return lon_min  <= input <= lon_max 

    def pressure_500(input):
        return input == 50000

    region_con = iris.Constraint(latitude = region_lat, longitude = region_long)
    pressure_con = iris.Constraint( air_pressure= pressure_500)
    date_con = iris.Constraint(time=lambda t: t.point == unit.date2num(model_day2, 'days since 1850-01-01 00:00:00','proleptic_gregorian'))


    # Subset the cube to the location of interest
    cube2_subset = cube2.extract(region_con & pressure_con & date_con)
    cube22 = cube2.extract(date_con)
    #find spatial average for each day
    coords = ('longitude', 'latitude')

    #sort cube by increasing latitude
    cube1_sorted = sorted(cube1, 'latitude')

    #apply weighting
    cosinelat_weights_large=iris.analysis.cartography.cosine_latitude_weights(cube1_sorted)
    cosinelat_weights_small=iris.analysis.cartography.cosine_latitude_weights(cube2_subset)
    longitude2 = cube2_subset.coord('longitude')
    latitude2 = cube2_subset.coord('latitude')

    latitude1b = cube1_sorted.coord('latitude')
    longitude1b = cube1_sorted.coord('longitude')
    
    #resample cube1 to make it smaller
    sample_points = [('longitude', longitude2.points),
                     ('latitude',  latitude2.points)]
    resampled_cube1 = cube1_sorted.interpolate(sample_points, iris.analysis.Linear())

    #create duplicate cube in style of cube1 so the correlation will work
    duplicate_cube2 = resampled_cube1.copy()
    duplicate_cube2.data = cube2_subset.data

    #replace its data with cube2 data
    duplicate_cube2.data = cube2_subset.data
    cube2_final = duplicate_cube2

    #convert units
    cube1_final = (resampled_cube1 /9.81)

    #calculate correlation

    #calculate correlation with model data
    correlation = iris.analysis.stats.pearsonr(cube1_final,cube2_final, weights=cosinelat_weights_small)
    print(correlation.data)
    p_corr_value = correlation.data

    p_corr_list.append(p_corr_value) 
    count = count + 1

0.7578226105293633
0.6840577452797375
0.3172758427631548
0.7241943944510418
0.4038573992615802
0.6810801788778237
0.6655925285983155
0.4383839088772084
0.6322517591614508
0.48935447432613255
0.5295611512826343
0.7356957069937355
0.6562129897147496
0.7012867432297025
0.7260355543913372
0.3958108745663552
0.17563150917078457
0.6030593519665509
0.4217268666061302
0.49311927564933766
0.5767284693255194
0.7811541857835057
0.5478431559716419
0.6389464166037575
0.7517754365586343
0.8135758409769674
0.5269648675823758
0.6148538250680506
0.38802431926275105
0.6783868741390486
0.6556029536618821
0.601867723831529
0.5481098145904991
0.6529358098069062
0.531070403042396
0.5466666360295862
0.4233377552555013
0.6594394541187961
0.6660237376774862
0.673686725587156
0.5785104960393941
0.39717178008299403
0.7009808381540349
0.4625144263340058
0.6642330998834274
0.2630984026328119
0.6945065271775555
0.672272860634415
0.5941885491600564
0.7041238408237683
0.048990003017226974
0.41251236336840724
0.427965

In [15]:
#make dataframe
#after all dates are done
model_series = pd.DataFrame({'dates':date_string_list,'member': member_list,'p_corr': p_corr_list}, columns=['dates','member','p_corr'])
model_series

dates      member               p_corr
0    2015-06-29  r112i1p1f1   0.7578226105293633
1    2032-06-28  r111i1p1f1   0.6840577452797375
2    2045-06-24  r105i1p1f1   0.3172758427631548
3    2063-06-28  r101i1p1f1   0.7241943944510418
4    2079-06-27  r103i1p1f1   0.4038573992615802
5    2082-06-30  r108i1p1f1   0.6810801788778237
6    2090-06-26  r108i1p1f1   0.6655925285983155
7    2091-06-26  r101i1p1f1   0.4383839088772084
8    2092-06-30  r107i1p1f1   0.6322517591614508
9    2094-06-30  r103i1p1f1  0.48935447432613255
10   2095-06-26  r112i1p1f1   0.5295611512826343
11   2096-06-27  r105i1p1f1   0.7356957069937355
12   2098-06-26  r105i1p1f1   0.6562129897147496
13   2098-06-26  r108i1p1f1   0.7012867432297025
14   2099-06-24  r102i1p1f1   0.7260355543913372
15   2099-06-29  r103i1p1f1   0.3958108745663552
16   2099-06-26  r110i1p1f1  0.17563150917078457
17   2015-07-27  r108i1p1f1   0.6030593519665509
18   2016-07-24  r107i1p1f1   0.4217268666061302
19   2029-07-21  r111i1p1f1  0.49311927564933766
20   2035-07-22  r109i1p1f1   0.5767284693255194
21   2036-07-23  r112i1p1f1   0.7811541857835057
22   2038-07-24  r111i1p1f1   0.5478431559716419
23   2040-07-16  r102i1p1f1   0.6389464166037575
24   2048-07-13  r106i1p1f1   0.7517754365586343
25   2049-07-03  r108i1p1f1   0.8135758409769674
26   2050-07-09  r109i1p1f1   0.5269648675823758
27   2051-07-13  r111i1p1f1   0.6148538250680506
28   2052-07-22  r110i1p1f1  0.38802431926275105
29   2054-07-25  r105i1p1f1   0.6783868741390486
..          ...         ...                  ...
81   2093-07-13  r102i1p1f1   0.7283151440962977
82   2093-07-23  r104i1p1f1  0.42590119602559123
83   2094-07-06  r103i1p1f1   0.5840744651903097
84   2094-07-29  r105i1p1f1   0.7628578342849451
85   2094-07-30  r107i1p1f1    0.553661043864245
86   2094-07-22  r109i1p1f1    0.743842392849713
87   2094-07-19  r111i1p1f1   0.5228656993402672
88   2095-07-31  r104i1p1f1   0.6297865062387696
89   2095-07-17  r109i1p1f1   0.7086872644567156
90   2095-07-17  r111i1p1f1   0.6309421914606023
91   2096-07-01  r102i1p1f1   0.7333415072425471
92   2096-07-14  r103i1p1f1   0.2807384757862153
93   2096-07-23  r104i1p1f1   0.6816124462254727
94   2096-07-31  r105i1p1f1   0.5312039707118944
95   2096-07-11  r108i1p1f1   0.3409311333847067
96   2096-07-10  r112i1p1f1   0.5823373332960703
97   2097-07-13  r101i1p1f1    0.716990187455054
98   2098-07-28  r104i1p1f1   0.6964489749064529
99   2098-07-17  r106i1p1f1   0.7747201930766537
100  2098-07-15  r108i1p1f1    0.697042656105868
101  2099-07-15  r103i1p1f1   0.6760562025758502
102  2099-07-14  r106i1p1f1  0.46880185793217033
103  2099-07-08  r107i1p1f1   0.5019349967953077
104  2099-07-20  r108i1p1f1   0.6049111470834879
105  2099-07-11  r110i1p1f1   0.6123634404154393
106  2099-07-27  r112i1p1f1   0.7258806791855811
107  2100-07-02  r101i1p1f1   0.2473970219059241
108  2100-07-08  r106i1p1f1   0.7367930164268043
109  2100-07-31  r107i1p1f1   0.6367481148107527
110  2100-07-21  r108i1p1f1  0.27705987778331664

[111 rows x 3 columns]

In [16]:
model_series.to_excel(output_name)